Iz html-jev sem slabo prebral datume, in sicer obcasno se pojavi, da je na lokaciji datuma zapisan drugi kraj, to se zgodi kadar imamo vec kot en kraj, jaz pa sem splital po prvi vejici in zato shranil drugi kraj na mesto datuma <br>  **POMEMBNO** vsi datumi niso popravljeni, a sem pripravljen reskirati nekaj clankov (priblizno 300 / 305000)

In [1]:
import re
from datetime import datetime
import json
import os

In [2]:
with open("../clanki_z_labelami.json", "r", encoding="utf-8") as f:
    clanki = json.load(f)

In [3]:
def is_date(text: str) -> bool:
    """
    Preveri, ali je podano besedilo datum, z ali brez časa.
    Podprti formati vključujejo:
    - DD.MM.YYYY
    - DD.MM.YY HH.MM
    - YYYY-MM-DD
    - DD/MM/YYYY
    - DD. mesec YYYY
    """

    text = text.strip()

    # Vnaprej definirani regularni izrazi
    date_patterns = [
        r"\b\d{1,2}\.\s*\d{1,2}\.\s*\d{4}\b",              # 28. 5. 2024
        r"\b\d{4}-\d{1,2}-\d{1,2}\b",                      # 2024-05-28
        r"\b\d{1,2}/\d{1,2}/\d{4}\b",                      # 28/05/2024
        r"\b\d{1,2}\.\s*[a-zA-ZčšžČŠŽ]+\s*\d{4}\b",        # 28. maj 2024
        r"\b\d{1,2}\.\d{1,2}\.\d{2}\s+\d{1,2}\.\d{1,2}\b", # 24.03.25 00.00
    ]

    for pattern in date_patterns:
        if re.search(pattern, text):
            return True

    # Seznam možnih datetime formatov
    possible_formats = [
        "%d.%m.%Y", "%d. %m. %Y", "%Y-%m-%d",
        "%d/%m/%Y", "%d. %B %Y", "%d. %b %Y",
        "%d.%m.%y %H.%M",  # 24.03.25 00.00
        "%d.%m.%Y %H.%M",
        "%Y-%m-%d %H:%M",
        "%d/%m/%Y %H:%M"
    ]

    for fmt in possible_formats:
        try:
            datetime.strptime(text, fmt)
            return True
        except ValueError:
            continue

    return False

In [4]:
i=0

for clanek in clanki:
    if not is_date(clanek.get("date", "").strip()):
        #print( f"{clanek.get('title', '')}")
        i=i+1

print(f"Vseh clankov z nepravilnim datumom je: {i}")

Vseh clankov z nepravilnim datumom je: 684


In [5]:
def odstrani_sumnike(besedilo: str) -> str:
    zamenjave = {
        "č": "c", "š": "s", "ž": "z",
        "Č": "c", "Š": "s", "Ž": "z"
    }
    return "".join(zamenjave.get(znak, znak) for znak in besedilo)

In [6]:
def najdi_html_pot(naslov: str) -> str | None:
    """
    Iz naslova ustvari ime HTML datoteke in preveri, če obstaja v ../../html/.

    Primer:
        "Slovenska politika danes" → "../html/slovenska-politika-danes.html"

    Vrne:
        Pot do datoteke, če obstaja, sicer None.
    """
    # Pretvori naslov v ime datoteke
    ime_datoteke = naslov.lower().replace(" ", "-")
    ime_datoteke = odstrani_sumnike(ime_datoteke)
    ime_datoteke = ime_datoteke.replace('"', '').replace("'", '').replace(",", "").replace("?","").replace(":", "").replace(".", "").replace("–", "-").replace("--", "-").replace("---", "-")
    ime_datoteke = ime_datoteke + ".html"
    pot = os.path.join("..","..", "html", ime_datoteke)

    # Preveri obstoj datoteke
    if os.path.exists(pot):
        return pot
    else:
        return None


In [7]:
i=0
for clanek in clanki:
    if not is_date(clanek.get("date", "").strip()):
        pot = najdi_html_pot(clanek.get("title",""))
        if pot:
            i=i+1
        #else:
            #print(clanek.get("title",""))

print(f"Najdenih je bilo {i} poti")

Najdenih je bilo 369 poti


Kljub veliko omejitvam vseh clankov nemorm vec povezati z njihovimi osnovnimi datotekami, zato sem se odlocil da je to sorazmerno majhen delez in bom nadaljeval brez njih

In [8]:
from bs4 import BeautifulSoup


In [9]:
def getDatum(soup):
    return soup.select_one('article > div > p').get_text(strip=True).split(",")[-1].split("|")[0]

    

In [10]:
i=0
for clanek in clanki:
    if not is_date(clanek.get("date", "").strip()):
        pot = najdi_html_pot(clanek.get("title",""))
        if pot:
            with open(pot, 'r', encoding='utf-8') as f:
                article=BeautifulSoup(f, 'html.parser')
            datum = getDatum(article)
            if is_date(datum):
                clanek["date"] = datum
                i=i+1
            else:
                print(datum)

print(f"Popravljenih je bilo {i} datumov")

 da je to njena dolžnost.
 nastopita popolna tišina in tema. Začne se lebdenje.
 na katerem so podirali rekorde. Pomerilo se je namreč 151 kosmatincev iz kar 39 različnih držav.
 da bi v domači Črni gori postal vinogradnik.
 se bodo ekstremni politični pojavi vrnili – močnejši kot kdaj koli prej."
 ki jim starši težko priskrbijo rojstnodnevno torto.
 kako izboljšati privlačnost Slovenije za tuje naložbe.
 je precepljenost še vedno nizka.
 z nami pa se je pogovarjala pred odprtjem razstave.
 ga še vedno preganja.
 zakaj so taki zagovorniki vrednot tudi zunaj igrišča.
 dražje in bolj boleče.
" pa pove 18-letno dekle. Oba sta uspela pobegniti vojni in zdaj živita v Sloveniji.
 da je za strah kriva ali slaba izkušnja ali nepravilna vzgoja.
 ki so jo prebili na zemlji svojih očetov.
 ki jih je ujel.
 kjer smo podelili nagrade za motošportnik leta. Na notranjepolitičnem področju je medtem vladala le ena tema – Nataša Pirc Musar je tudi uradno postala prva predsednica Slovenije. Kateri foto u

In [11]:
with open("combined_finish.json", 'w', encoding='utf-8') as f:
    json.dump(clanki, f, ensure_ascii=False, indent=2)